pip install numpy  
pip install pandas  
pip install openpyxl  
pip install tqdm
pip install ffmpeg  
pip install pydub  
conda install -c conda-forge librosa  

In [1]:
import pydub
import pandas as pd
from tqdm import tqdm

pydub.AudioSegment.converter = r"C:/Users/Kinza/anaconda3/envs/pie/Library/bin/ffmpeg.exe" #CHANGE THIS

# Data loading

In [2]:
def get_end_from_start(df_mail):
    end             = df_mail.iloc[1:,:]['start'].copy()
    end[len(end)+1] = -1
    df_mail['end']  = end.values
    return df_mail

def get_start_end_from_file(file):
    #Get start and end times
    df = pd.read_excel(file)
    df = df.rename(columns={'time':'start'}) 
    df['start'] = df['start']*1000 #in ms

    df = df.groupby('mail').apply(get_end_from_start)
    return df

def load_audios(video_folder,startend_file) :
    '''Loads the audios from each interview question.
    Arguments:
        video_folder : str. The name of the folder containing mp4 videos.
        startend_file : str. The name of the file containing the video informations. 
            Must contain columns 'mail' and 'time'.
    Returns a list of audios
    '''
    filenames = tqdm(os.listdir(video_folder))
    df_startend = get_start_end_from_file(startend_file)
    audios = list(map(lambda f : split_questions(video_folder,df_startend,f), filenames))
    return [item for sublist in audios for item in sublist]


In [3]:
from audio_utils import split_questions

video_folder = 'videos/'
df_name = 'test.xlsx'

audios = load_audios(video_folder,df_name)

100%|██████████| 4/4 [00:06<00:00,  1.63s/it]


# Preprocessing

In [4]:
#silence_feats = list(map(pauses_features,tqdm(audios)))[:-1]
from audio_feats_extract import Audio
audios = [Audio(audio) for audio in audios]

In [5]:
for audio in tqdm(audios):
    audio.preprocessing()

100%|██████████| 12/12 [00:31<00:00,  2.65s/it]


spectral_features.columns = ['chroma_stft','rms','spec_cent','spec_bw','rolloff','zcr']+['mfcc_'+str(i) for i in range(20)]

# Prosodic features

It captures the intonation of speech, the rhythm or the tone of speech. They reveal the information
about the identity, attitude and emotional state of the underlying signal

# Load features

In [ ]:
import pandas as pd

features = pd.read_csv('features.csv',index_col=0)
features.index = map(lambda s : s.split('@')[0],filenames)

# PCA

In [ ]:
from pca import pca

model = pca(n_components=2)

# Fit transform
features_pca = model.fit_transform(features.drop('classes',axis=1))

print(model.compute_topfeat())

# Scatter first 2 PCs
fig, ax = model.scatter(legend=False)

# Make biplot with the number of features
fig, ax = model.biplot(n_feat=10,legend=False)

import matplotlib.pyplot as plt
plt.scatter(features_pca['PC']['PC1'],features_pca['PC']['PC2'],c=features['classes'])